In [1]:
import pandas as pd

In [4]:
test_actual_path = "data/EN/test.txt"
test_predicted_path = "results/output.txt"

In [5]:
with open(test_actual_path,'r') as f:
    test_actual = f.readlines()

In [6]:
with open(test_predicted_path,'r') as f:
    test_pred = f.readlines()

In [7]:
def sent_extractor(collection):
        sents = []
        tok_det = []
        for i in collection:
                if i=="\n":
                    sents.append(tok_det)
                    tok_det = []
                else:
                    tok_det.append(i)

        if len(tok_det) > 0:
            sents.append(tok_det)
        
        return sents

In [8]:
def read(sents):
		seqs = []
		for i in sents:
			seqs_i = []
			for j in i:
				j_list = j.split('\t')
				seqs_i.append([j_list[0], j_list[1], j_list[7]])
			seqs.append(seqs_i)
		
		return seqs

In [9]:
test_actual_sent = sent_extractor(test_actual) 
test_pred_sent = sent_extractor(test_pred)

In [10]:
test_actual_tags = read(test_actual_sent)

In [11]:
test_pred_tags = read(test_pred_sent)

In [12]:
def mwe_tag_index_extractor(tags):
    mwe_tags_start = {}
    for idx, sentence in enumerate(tags):
        print("Working on index {}".format(idx))
        mwe_tags_start[idx] = []
        for i,j in enumerate(sentence):
            mwe_indices = []
            if j[2] == 'B':
                print("------- Found B-tag at {}".format(i))
                k = i
                mwe_indices.append(k)
                if k+1 < len(sentence):
                    k += 1
                else:
                    break
                while sentence[k][2] == 'I':
                    print("---------------- Appended k: {}".format(k))
                    mwe_indices.append(k)
                    k += 1
                    if k >= len(sentence):
                        break
                mwe_tags_start[idx].append(mwe_indices)
    return mwe_tags_start

In [13]:
test_actual_mwe_tags = mwe_tag_index_extractor(test_actual_tags)

Working on index 0
------- Found B-tag at 7
---------------- Appended k: 8
---------------- Appended k: 9
------- Found B-tag at 17
---------------- Appended k: 18
------- Found B-tag at 24
---------------- Appended k: 25
---------------- Appended k: 26
------- Found B-tag at 28
---------------- Appended k: 29
------- Found B-tag at 31
---------------- Appended k: 32
Working on index 1
------- Found B-tag at 0
---------------- Appended k: 1
------- Found B-tag at 3
---------------- Appended k: 4
---------------- Appended k: 5
---------------- Appended k: 6
---------------- Appended k: 7
---------------- Appended k: 8
---------------- Appended k: 9
---------------- Appended k: 10
---------------- Appended k: 11
Working on index 2
------- Found B-tag at 1
---------------- Appended k: 2
---------------- Appended k: 3
------- Found B-tag at 11
---------------- Appended k: 12
Working on index 3
------- Found B-tag at 4
---------------- Appended k: 5
---------------- Appended k: 6
------- Fo

---------------- Appended k: 8
------- Found B-tag at 21
---------------- Appended k: 22
---------------- Appended k: 23
Working on index 743
------- Found B-tag at 1
---------------- Appended k: 2
Working on index 744
------- Found B-tag at 10
---------------- Appended k: 11
------- Found B-tag at 15
---------------- Appended k: 16
---------------- Appended k: 17
Working on index 745
------- Found B-tag at 7
---------------- Appended k: 8
---------------- Appended k: 9
------- Found B-tag at 14
---------------- Appended k: 15
Working on index 746
------- Found B-tag at 3
---------------- Appended k: 4
---------------- Appended k: 5
------- Found B-tag at 9
---------------- Appended k: 10
Working on index 747
------- Found B-tag at 24
---------------- Appended k: 25
Working on index 748
------- Found B-tag at 0
---------------- Appended k: 1
---------------- Appended k: 2
---------------- Appended k: 3
---------------- Appended k: 4
---------------- Appended k: 5
---------------- Appen

---------------- Appended k: 30
Working on index 1437
------- Found B-tag at 1
---------------- Appended k: 2
------- Found B-tag at 12
---------------- Appended k: 13
Working on index 1438
------- Found B-tag at 5
---------------- Appended k: 6
---------------- Appended k: 7
Working on index 1439
------- Found B-tag at 3
------- Found B-tag at 11
---------------- Appended k: 12
---------------- Appended k: 13
------- Found B-tag at 17
---------------- Appended k: 18
---------------- Appended k: 19
---------------- Appended k: 20
Working on index 1440
------- Found B-tag at 7
---------------- Appended k: 8
Working on index 1441
------- Found B-tag at 4
---------------- Appended k: 5
------- Found B-tag at 8
---------------- Appended k: 9
------- Found B-tag at 26
---------------- Appended k: 27
---------------- Appended k: 28
------- Found B-tag at 40
---------------- Appended k: 41
------- Found B-tag at 60
---------------- Appended k: 61
------- Found B-tag at 78
---------------- App

Working on index 2186
------- Found B-tag at 1
---------------- Appended k: 2
---------------- Appended k: 3
------- Found B-tag at 21
---------------- Appended k: 22
Working on index 2187
------- Found B-tag at 1
---------------- Appended k: 2
------- Found B-tag at 9
---------------- Appended k: 10
---------------- Appended k: 11
------- Found B-tag at 21
---------------- Appended k: 22
---------------- Appended k: 23
------- Found B-tag at 33
---------------- Appended k: 34
Working on index 2188
------- Found B-tag at 8
---------------- Appended k: 9
Working on index 2189
------- Found B-tag at 1
---------------- Appended k: 2
------- Found B-tag at 22
---------------- Appended k: 23
Working on index 2190
------- Found B-tag at 4
---------------- Appended k: 5
Working on index 2191
------- Found B-tag at 5
---------------- Appended k: 6
---------------- Appended k: 7
---------------- Appended k: 8
------- Found B-tag at 9
---------------- Appended k: 10
------- Found B-tag at 13
---

In [14]:
test_pred_mwe_tags = mwe_tag_index_extractor(test_pred_tags)

Working on index 0
------- Found B-tag at 7
---------------- Appended k: 8
---------------- Appended k: 9
------- Found B-tag at 17
---------------- Appended k: 18
------- Found B-tag at 24
---------------- Appended k: 25
---------------- Appended k: 26
------- Found B-tag at 28
---------------- Appended k: 29
------- Found B-tag at 31
---------------- Appended k: 32
Working on index 1
------- Found B-tag at 0
---------------- Appended k: 1
------- Found B-tag at 3
---------------- Appended k: 4
---------------- Appended k: 5
---------------- Appended k: 6
---------------- Appended k: 7
---------------- Appended k: 8
---------------- Appended k: 9
---------------- Appended k: 10
---------------- Appended k: 11
Working on index 2
------- Found B-tag at 1
---------------- Appended k: 2
---------------- Appended k: 3
------- Found B-tag at 11
---------------- Appended k: 12
---------------- Appended k: 13
Working on index 3
------- Found B-tag at 4
---------------- Appended k: 5
---------

---------------- Appended k: 20
Working on index 758
------- Found B-tag at 9
---------------- Appended k: 10
------- Found B-tag at 20
---------------- Appended k: 21
---------------- Appended k: 22
Working on index 759
------- Found B-tag at 81
---------------- Appended k: 82
---------------- Appended k: 83
------- Found B-tag at 91
---------------- Appended k: 92
---------------- Appended k: 93
Working on index 760
------- Found B-tag at 17
---------------- Appended k: 18
------- Found B-tag at 52
---------------- Appended k: 53
Working on index 761
------- Found B-tag at 20
---------------- Appended k: 21
------- Found B-tag at 28
---------------- Appended k: 29
------- Found B-tag at 32
---------------- Appended k: 33
Working on index 762
------- Found B-tag at 13
---------------- Appended k: 14
---------------- Appended k: 15
---------------- Appended k: 16
------- Found B-tag at 20
---------------- Appended k: 21
------- Found B-tag at 53
---------------- Appended k: 54
Working 

------- Found B-tag at 16
---------------- Appended k: 17
Working on index 1489
------- Found B-tag at 2
---------------- Appended k: 3
---------------- Appended k: 4
------- Found B-tag at 6
---------------- Appended k: 7
Working on index 1490
------- Found B-tag at 2
---------------- Appended k: 3
---------------- Appended k: 4
Working on index 1491
------- Found B-tag at 1
---------------- Appended k: 2
Working on index 1492
------- Found B-tag at 6
---------------- Appended k: 7
---------------- Appended k: 8
------- Found B-tag at 30
---------------- Appended k: 31
---------------- Appended k: 32
------- Found B-tag at 33
---------------- Appended k: 34
---------------- Appended k: 35
---------------- Appended k: 36
------- Found B-tag at 49
---------------- Appended k: 50
------- Found B-tag at 58
---------------- Appended k: 59
------- Found B-tag at 63
---------------- Appended k: 64
---------------- Appended k: 65
---------------- Appended k: 66
Working on index 1493
------- F

---------------- Appended k: 18
---------------- Appended k: 19
Working on index 2225
------- Found B-tag at 28
---------------- Appended k: 29
Working on index 2226
------- Found B-tag at 9
---------------- Appended k: 10
------- Found B-tag at 17
---------------- Appended k: 18
---------------- Appended k: 19
------- Found B-tag at 21
---------------- Appended k: 22
Working on index 2227
------- Found B-tag at 4
---------------- Appended k: 5
---------------- Appended k: 6
------- Found B-tag at 15
---------------- Appended k: 16
Working on index 2228
------- Found B-tag at 8
---------------- Appended k: 9
Working on index 2229
------- Found B-tag at 1
---------------- Appended k: 2
---------------- Appended k: 3
---------------- Appended k: 4
Working on index 2230
------- Found B-tag at 6
---------------- Appended k: 7
Working on index 2231
------- Found B-tag at 1
---------------- Appended k: 2
------- Found B-tag at 4
---------------- Appended k: 5
---------------- Appended k: 6
-

In [15]:
sum([len(v) for k,v in test_actual_mwe_tags.items()]) 

4825

In [16]:
sum([len(v) for k,v in test_pred_mwe_tags.items()])

4804

## True positive - Match found.
## True Negative - not in test set, not predicted
## False positive - Not in actual test set, but predicted
## False Negative - in the actual test set, but not predicted

In [17]:
def metrics(test_actual_mwe_tags, test_pred_mwe_tags):
    exact_matches_tp = 0
    exact_matches_fn = 0
    exact_matches_fp = 0
    tp_list = []
    fp_list = []
    fn_list = []
    for k,v in test_actual_mwe_tags.items():
        test_actual_tag_indices = v
        test_pred_tag_indices = test_pred_mwe_tags[k]
        print(k)
        print(test_actual_tag_indices)
        print(test_pred_tag_indices)
        exact_match_tp = 0
        exact_match_fn = 0
        exact_match_fp = 0
        for i in test_actual_tag_indices:
            print("Searching for a match for {}".format(i))
            if i in test_pred_tag_indices:
                print("True positive")
                exact_match_tp += 1
            else:
                print("True Negative")
                exact_match_fn += 1

        for i in test_pred_tag_indices:
            print("Searching for a match for {} in the actual test".format(i))
            if i not in test_actual_tag_indices:
                print("False positive")
                exact_match_fp += 1

        tp_list.append(exact_match_tp)
        fp_list.append(exact_match_fp)
        fn_list.append(exact_match_fn)

        exact_matches_tp += exact_match_tp
        exact_matches_fn += exact_match_fn
        exact_matches_fp += exact_match_fp
        
    return exact_matches_tp, exact_matches_fn, exact_matches_fp


In [18]:
tp, fn, fp = metrics(test_actual_mwe_tags, test_pred_mwe_tags)

0
[[7, 8, 9], [17, 18], [24, 25, 26], [28, 29], [31, 32]]
[[7, 8, 9], [17, 18], [24, 25, 26], [28, 29], [31, 32]]
Searching for a match for [7, 8, 9]
True positive
Searching for a match for [17, 18]
True positive
Searching for a match for [24, 25, 26]
True positive
Searching for a match for [28, 29]
True positive
Searching for a match for [31, 32]
True positive
Searching for a match for [7, 8, 9] in the actual test
Searching for a match for [17, 18] in the actual test
Searching for a match for [24, 25, 26] in the actual test
Searching for a match for [28, 29] in the actual test
Searching for a match for [31, 32] in the actual test
1
[[0, 1], [3, 4, 5, 6, 7, 8, 9, 10, 11]]
[[0, 1], [3, 4, 5, 6, 7, 8, 9, 10, 11]]
Searching for a match for [0, 1]
True positive
Searching for a match for [3, 4, 5, 6, 7, 8, 9, 10, 11]
True positive
Searching for a match for [0, 1] in the actual test
Searching for a match for [3, 4, 5, 6, 7, 8, 9, 10, 11] in the actual test
2
[[1, 2, 3], [11, 12]]
[[1, 2, 3],

Searching for a match for [1]
True Negative
Searching for a match for [13, 14, 15]
True positive
Searching for a match for [13, 14, 15] in the actual test
494
[[4, 5], [8, 9], [10, 11], [17, 18], [29, 30, 31], [39, 40, 41, 42, 43]]
[[4, 5], [8, 9], [10, 11], [17, 18], [29, 30, 31], [39, 40, 41, 42, 43]]
Searching for a match for [4, 5]
True positive
Searching for a match for [8, 9]
True positive
Searching for a match for [10, 11]
True positive
Searching for a match for [17, 18]
True positive
Searching for a match for [29, 30, 31]
True positive
Searching for a match for [39, 40, 41, 42, 43]
True positive
Searching for a match for [4, 5] in the actual test
Searching for a match for [8, 9] in the actual test
Searching for a match for [10, 11] in the actual test
Searching for a match for [17, 18] in the actual test
Searching for a match for [29, 30, 31] in the actual test
Searching for a match for [39, 40, 41, 42, 43] in the actual test
495
[[8, 9], [38, 39]]
[[8, 9], [38, 39]]
Searching f

Searching for a match for [6, 7] in the actual test
957
[[2, 3], [4, 5], [6, 7], [16, 17], [19, 20], [28, 29], [37, 38], [59, 60]]
[[2, 3], [4, 5], [6, 7], [16, 17], [19, 20], [28, 29], [37, 38], [59, 60]]
Searching for a match for [2, 3]
True positive
Searching for a match for [4, 5]
True positive
Searching for a match for [6, 7]
True positive
Searching for a match for [16, 17]
True positive
Searching for a match for [19, 20]
True positive
Searching for a match for [28, 29]
True positive
Searching for a match for [37, 38]
True positive
Searching for a match for [59, 60]
True positive
Searching for a match for [2, 3] in the actual test
Searching for a match for [4, 5] in the actual test
Searching for a match for [6, 7] in the actual test
Searching for a match for [16, 17] in the actual test
Searching for a match for [19, 20] in the actual test
Searching for a match for [28, 29] in the actual test
Searching for a match for [37, 38] in the actual test
Searching for a match for [59, 60] i

Searching for a match for [39, 40, 41, 42]
True positive
Searching for a match for [20, 21] in the actual test
Searching for a match for [39, 40, 41, 42] in the actual test
1436
[[28, 29, 30]]
[[28, 29]]
Searching for a match for [28, 29, 30]
True Negative
Searching for a match for [28, 29] in the actual test
False positive
1437
[[1, 2], [12, 13]]
[[1, 2], [12, 13]]
Searching for a match for [1, 2]
True positive
Searching for a match for [12, 13]
True positive
Searching for a match for [1, 2] in the actual test
Searching for a match for [12, 13] in the actual test
1438
[[5, 6, 7]]
[[5, 6, 7], [22, 23, 24]]
Searching for a match for [5, 6, 7]
True positive
Searching for a match for [5, 6, 7] in the actual test
Searching for a match for [22, 23, 24] in the actual test
False positive
1439
[[3], [11, 12, 13], [17, 18, 19, 20]]
[[3], [11, 12, 13], [17, 18, 19, 20]]
Searching for a match for [3]
True positive
Searching for a match for [11, 12, 13]
True positive
Searching for a match for [17,

Searching for a match for [47, 48] in the actual test
1934
[[1, 2, 3], [5, 6]]
[[1, 2, 3], [5, 6], [8, 9, 10]]
Searching for a match for [1, 2, 3]
True positive
Searching for a match for [5, 6]
True positive
Searching for a match for [1, 2, 3] in the actual test
Searching for a match for [5, 6] in the actual test
Searching for a match for [8, 9, 10] in the actual test
False positive
1935
[[2, 3, 4, 5], [12, 13], [27, 28]]
[[2, 3, 4, 5], [12, 13], [27, 28]]
Searching for a match for [2, 3, 4, 5]
True positive
Searching for a match for [12, 13]
True positive
Searching for a match for [27, 28]
True positive
Searching for a match for [2, 3, 4, 5] in the actual test
Searching for a match for [12, 13] in the actual test
Searching for a match for [27, 28] in the actual test
1936
[[0, 1]]
[[0, 1]]
Searching for a match for [0, 1]
True positive
Searching for a match for [0, 1] in the actual test
1937
[[0, 1, 2], [6, 7, 8, 9]]
[[0, 1, 2], [6, 7, 8, 9]]
Searching for a match for [0, 1, 2]
True po

In [19]:
tp, fn, fp

(4300, 525, 504)

In [20]:
rec = tp / (tp+fn)

In [21]:
pre = tp / (tp+fp)

In [22]:
rec, pre

(0.8911917098445595, 0.8950874271440467)

In [23]:
f1 = 2 / ((1.0/rec) + (1.0/pre))

In [24]:
f1

0.893135320386333